<a href="https://colab.research.google.com/github/SvA1/Compling/blob/master/HW5_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 7.1MB 14.2MB/s 


In [3]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 9.6MB/s 
     |████████████████████████████████| 552kB 45.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=88935fd6a7ec365e78ab210d215a4bd15f47f113544353cf1e09cbc50a17dd06
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=4c0c972212b4e19228fe4f0f869a0b0bb5967f0b2ea9b425485aef2ef90157ff
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import re
import json
import gensim
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter

morph = MorphAnalyzer()

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/habr_texts.txt.zip
!unzip habr_texts.txt.zip

In [0]:
with open('habr_texts.txt', encoding='utf-8') as file:
    source = file.read().splitlines()

**1. Нормализация**

In [0]:
stop_words = set(stopwords.words('russian')) | {'gt',}

def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'{[^}]+}', ' ', text)
    text = re.sub(r'\&[a-z]+\;', ' ', text)
    text = text.replace('\t', ' ')
    return text

def tokenize(text):
    return re.findall(r'[А-ЯЁа-яё]+', text)

In [0]:
def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stop_words]
        norm_texts.append(norm_words)
        
    return norm_texts

**2. Добавление нграммов**

In [0]:
texts = opt_normalize([tokenize(clean_text(text)) for text in source])

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', min_count=8, threshold=0.2)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

**3. Словарь**

In [0]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [0]:
dictionary.filter_extremes(no_above=0.1, no_below=15)
dictionary.compactify()

In [0]:
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

**4. LDA модели**

In [0]:
lda1 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=5, passes=2, alpha='asymmetric')
lda2 = gensim.models.LdaModel(corpus, id2word=dictionary, eval_every=1, num_topics=10, passes=3, alpha='auto')
lda3 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=2, num_topics=20, passes=5)

In [15]:
lda1.print_topics(15)

[(0,
  '0.003*"дефект" + 0.003*"раздел" + 0.002*"массив" + 0.002*"логика" + 0.002*"вычисление" + 0.002*"ядро" + 0.002*"переменный" + 0.002*"атрибут" + 0.002*"китай" + 0.002*"кожа"'),
 (1,
  '0.004*"студент" + 0.003*"цикл" + 0.003*"домен" + 0.003*"бот" + 0.003*"прибор" + 0.003*"узел" + 0.002*"процессор" + 0.002*"сигнал" + 0.002*"температура" + 0.002*"курс"'),
 (2,
  '0.005*"игрок" + 0.005*"боль" + 0.004*"массив" + 0.003*"браузер" + 0.003*"репликация" + 0.003*"робот" + 0.003*"дом" + 0.002*"стикер" + 0.002*"электроника" + 0.002*"играть"'),
 (3,
  '0.006*"услуга" + 0.003*"заказчик" + 0.003*"инцидент" + 0.003*"контейнер" + 0.002*"мозг" + 0.002*"лицо" + 0.002*"атрибут" + 0.002*"поль" + 0.002*"сборка" + 0.002*"офис"'),
 (4,
  '0.005*"игрок" + 0.004*"вакансия" + 0.003*"кампания" + 0.003*"доклад" + 0.002*"браузер" + 0.002*"дерево" + 0.002*"контент" + 0.002*"контекст" + 0.002*"объявление" + 0.002*"клетка"')]

In [16]:
lda2.print_topics(10)

[(0,
  '0.006*"доклад" + 0.005*"звук" + 0.004*"конференция" + 0.004*"публикация" + 0.004*"цифровой" + 0.003*"усилитель" + 0.003*"реклама" + 0.003*"синтезатор" + 0.003*"ия" + 0.002*"продажа"'),
 (1,
  '0.006*"яркость" + 0.006*"координата" + 0.006*"прибор" + 0.005*"аккумулятор" + 0.005*"сигнал" + 0.004*"стандарт" + 0.004*"корпус" + 0.004*"свет" + 0.004*"смартфон" + 0.003*"телефон"'),
 (2,
  '0.012*"мозг" + 0.010*"головка" + 0.010*"клетка" + 0.009*"печать" + 0.008*"учёный" + 0.007*"лицо" + 0.007*"кожа" + 0.006*"принтер" + 0.006*"ребёнок" + 0.005*"температура"'),
 (3,
  '0.021*"боль" + 0.006*"сигнал" + 0.006*"сопротивление" + 0.005*"дом" + 0.005*"ткань" + 0.004*"изделие" + 0.004*"пациент" + 0.004*"заказчик" + 0.004*"датчик" + 0.004*"температура"'),
 (4,
  '0.005*"курс" + 0.004*"лекция" + 0.004*"программирование" + 0.004*"веб" + 0.004*"перевод" + 0.004*"браузер" + 0.003*"подборка" + 0.003*"студент" + 0.003*"гибкость" + 0.003*"ядро"'),
 (5,
  '0.019*"услуга" + 0.007*"инцидент" + 0.005*"трафи

In [17]:
lda3.print_topics(20)

[(0,
  '0.006*"головка" + 0.005*"дом" + 0.005*"температура" + 0.004*"смартфон" + 0.004*"цифровой" + 0.003*"принтер" + 0.003*"датчик" + 0.003*"кабель" + 0.003*"печать" + 0.003*"расчёт"'),
 (1,
  '0.034*"услуга" + 0.009*"студент" + 0.007*"южный_корея" + 0.006*"программирование" + 0.006*"инфраструктура" + 0.005*"математика" + 0.005*"облако" + 0.005*"ответственный" + 0.005*"отметка" + 0.005*"искусственный_интеллект"'),
 (2,
  '0.009*"студент" + 0.007*"курс" + 0.004*"квеста" + 0.004*"китай" + 0.004*"лекция" + 0.004*"синтезатор" + 0.003*"аудитория" + 0.003*"товар" + 0.003*"помещение" + 0.003*"загадка"'),
 (3,
  '0.009*"электроника" + 0.008*"олимпиада" + 0.005*"земля" + 0.004*"российский" + 0.004*"аппарат" + 0.004*"школьник" + 0.004*"закон" + 0.004*"корпорация" + 0.004*"физика" + 0.004*"спутник"'),
 (4,
  '0.025*"игрок" + 0.009*"играть" + 0.006*"игровой" + 0.005*"респондент" + 0.004*"аудитория" + 0.003*"ваш_игра" + 0.003*"дом" + 0.003*"кейс" + 0.003*"текстура" + 0.003*"наблюдать"'),
 (5,
  '0

**5. 3 темы для модели**

Машинное обучение

>   '0.017*"массив" + 0.011*"цикл" + 0.008*"репликация" + 0.006*"переменный" + 0.006*"печать" + 0.006*"кожа" + 0.005*"матрица" + 0.005*"вектор" + 0.004*"выполняться" + 0.003*"дерево"'),

Информационная безопасность??(возможно, что-то связанное с видеонаблюдением)

>   '0.012*"камера" + 0.010*"датчик" + 0.007*"заказчик" + 0.005*"протокол" + 0.005*"шлюз" + 0.005*"прибор" + 0.005*"изделие" + 0.004*"инфраструктура" + 0.004*"взлом" + 0.004*"защитить"'),

Чат-боты
>  '0.015*"браузер" + 0.010*"доклад" + 0.010*"веб" + 0.007*"кэш" + 0.007*"токен" + 0.007*"контент" + 0.007*"бд" + 0.006*"контекст" + 0.005*"сущность" + 0.005*"бот"'

 




**6. TfIdf**

In [0]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus_tfidf = tfidf[corpus]

In [0]:
lda1_tfidf = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, eval_every=1, num_topics=5, passes=2, alpha='asymmetric')
lda2_tfidf = gensim.models.LdaModel(corpus_tfidf, id2word=dictionary, eval_every=1, num_topics=10, passes=3, alpha='auto')
lda3_tfidf = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, eval_every=2, num_topics=20, passes=5)

In [21]:
lda1_tfidf.print_topics(5)

[(0,
  '0.001*"браузер" + 0.001*"услуга" + 0.001*"игрок" + 0.001*"доклад" + 0.001*"контейнер" + 0.001*"заказчик" + 0.001*"контекст" + 0.001*"программирование" + 0.001*"контент" + 0.001*"бот"'),
 (1,
  '0.000*"стакан" + 0.000*"массив" + 0.000*"транзакция" + 0.000*"переменный" + 0.000*"бот" + 0.000*"пружина" + 0.000*"конференция" + 0.000*"смартфон" + 0.000*"плагин" + 0.000*"сборка"'),
 (2,
  '0.005*"битрикс" + 0.000*"метрика" + 0.000*"ознакомительный" + 0.000*"пациент" + 0.000*"знание_навык" + 0.000*"версия_продукт" + 0.000*"вопрос_связанный" + 0.000*"напряжение" + 0.000*"перенос" + 0.000*"конфига"'),
 (3,
  '0.000*"умный_дом" + 0.000*"робот" + 0.000*"сообщество" + 0.000*"камера" + 0.000*"одиночество" + 0.000*"контроллер" + 0.000*"трафик" + 0.000*"дом" + 0.000*"новость" + 0.000*"словарь"'),
 (4,
  '0.000*"видеокарта" + 0.000*"атака" + 0.000*"игровой" + 0.000*"игрок" + 0.000*"панель_инструмент" + 0.000*"репликация" + 0.000*"марс" + 0.000*"пк" + 0.000*"корейский" + 0.000*"ботнет"')]

In [22]:
lda2_tfidf.print_topics(10)

[(0,
  '0.001*"протез" + 0.000*"мастер_класс" + 0.000*"кружок" + 0.000*"кружка" + 0.000*"билет" + 0.000*"сатурн" + 0.000*"ученик" + 0.000*"москва_пройти" + 0.000*"наушник" + 0.000*"типичный_пример"'),
 (1,
  '0.002*"услуга" + 0.001*"заказчик" + 0.001*"трафик" + 0.001*"пароль" + 0.001*"студент" + 0.001*"инцидент" + 0.001*"домен" + 0.001*"робот" + 0.001*"вакансия" + 0.001*"уязвимость"'),
 (2,
  '0.006*"паяльник" + 0.004*"шт" + 0.002*"вт" + 0.001*"резистор" + 0.001*"пайка" + 0.001*"микроконтроллер" + 0.001*"печатный_плата" + 0.001*"датчик_температура" + 0.001*"конденсатор" + 0.001*"температура"'),
 (3,
  '0.007*"фсб" + 0.006*"законопроект" + 0.001*"вменяемый" + 0.001*"сим_карта" + 0.001*"мессенджер" + 0.000*"конвертер" + 0.000*"законодательство" + 0.000*"место_проведение" + 0.000*"мкс" + 0.000*"предоставление_услуга"'),
 (4,
  '0.004*"контейнер" + 0.004*"атрибут" + 0.003*"шаблон" + 0.003*"переменный" + 0.003*"контекст" + 0.003*"мастер_класс" + 0.003*"стиль" + 0.003*"южный_корея" + 0.002*"

In [23]:
lda3_tfidf.print_topics(20)

[(0,
  '0.005*"мессенджер" + 0.004*"дефект" + 0.004*"бот" + 0.004*"слайд" + 0.003*"торрент" + 0.003*"робот" + 0.003*"социальный" + 0.003*"социальный_сеть" + 0.003*"звук" + 0.003*"аудио"'),
 (1,
  '0.017*"репликация" + 0.013*"массив" + 0.010*"ускоритель" + 0.004*"присваивание" + 0.004*"рис" + 0.003*"пользовательский_опыт" + 0.003*"причудливый" + 0.003*"предоставление_услуга" + 0.003*"вовлечение" + 0.003*"отказ"'),
 (2,
  '0.015*"боль" + 0.011*"чек_лист" + 0.005*"принятие" + 0.005*"заглядывать" + 0.005*"подборка" + 0.005*"делиться_свой" + 0.005*"коммерсант" + 0.004*"информационный_безопасность" + 0.004*"казино" + 0.004*"тематика"'),
 (3,
  '0.010*"прямоугольник" + 0.009*"хук" + 0.006*"паблик" + 0.006*"личный_сообщение" + 0.004*"коммерческий_использование" + 0.004*"пересекать" + 0.003*"ёмкий" + 0.003*"вложить" + 0.003*"услышать" + 0.003*"инстанция"'),
 (4,
  '0.008*"двигатель" + 0.007*"прибор" + 0.006*"олимпиада" + 0.006*"мастер_класс" + 0.005*"флешек" + 0.005*"раздел" + 0.004*"законопрое

**7. Анализ**

Замметила, что в tfidf моделях извлеклись буквы как темы.


```
  '0.000*"з" + 0.000*"ребро" + 0.000*"ландшафт" + 0.000*"ки"
```

Непонятно, почему темы в третьей модели lda и в третьей модели tfidf настолько разнятся 

```
[(0,
  '0.010*"боль" + 0.006*"взлом" + 0.005*"продакт_менеджер" + 0.005*"сделка" + 0.005*"инвестор" + 0.004*"корпорация" + 0.004*"инвестиция" + 0.004*"принудительный" + 0.004*"учётный_запись" + 0.004*"исполнительный_директор"'),

[(0,
  '0.009*"дефект" + 0.008*"стикер" + 0.005*"граница" + 0.005*"гипотеза" + 0.004*"доска" + 0.004*"выражение" + 0.004*"гибкость" + 0.004*"шаблон" + 0.004*"рис" + 0.003*"хабра"'),

```

То же самое происходит и с первыми моделями

(2,
  '0.000*"генератор" + 0.000*"коммит" + 0.000*"уравнение" + 0.000*"дисплей" + 0.000*"сигнал" + 0.000*"звезда" + 0.000*"яйцо" + 0.000*"криптография" + 0.000*"функция_выполнять" + 0.000*"образование"')

(2,
  '0.004*"браузер" + 0.003*"кампания" + 0.002*"партнёр" + 0.002*"головка" + 0.002*"конвертер" + 0.002*"перевод" + 0.002*"шаблон" + 0.002*"ключ" + 0.002*"процессор" + 0.002*"трафик"'),


**8. NMF + Tfidf Vectorizer**

In [0]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [0]:
str_texts = [' '.join(text) for text in texts]

In [0]:
vectorizer = TfidfVectorizer(max_features=800, min_df=12, max_df=0.4, ngram_range=(1,2))
X = vectorizer.fit_transform(str_texts)

In [0]:
model = NMF(n_components=30)

In [28]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [0]:
feat_names = vectorizer.get_feature_names()

In [31]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 ваш  знать  сказать  говорить
1 файл  папка  скрипт  команда
2 игра  игрок  игровой  играть
3 функция  значение  переменный  массив
4 устройство  смартфон  телефон  производитель
5 приложение  мобильный  разработка  платформа
6 дата центр  центр  дата  оборудование
7 рубль  товар  цена  рынок
8 учёный  мозг  исследование  научный
9 уязвимость  атака  безопасность  защита
10 сайт  страница  браузер  реклама
11 доклад  конференция  разработчик  видео
12 сеть  интернет  трафик  связь
13 код  библиотека  ошибка  исходный
14 книга  часы  читать  электронный
15 запрос  база  таблица  база дать
16 язык  программирование  программист  программа
17 объект  класс  метод  тип
18 элемент  компонент  блок  свойство
19 память  процессор  диск  ядро
20 тест  тестирование  тестовый  производительность
21 сигнал  звук  частота  диапазон
22 камера  видео  изображение  смартфон
23 робот  автомобиль  машина  ребёнок
24 сервер  виртуальный  машина  настройка
25 бот  сообщение  канал  команда
26 ключ  сер

In [32]:
vectorizer = TfidfVectorizer(max_features=500, min_df=7, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(str_texts)
model = NMF(n_components=40)
model.fit(X)
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 жизнь  друг  думать  вообще
1 игра  игрок  игровой  играть
2 файл  строка  пароль  формат
3 сервер  домен  настройка  адрес
4 устройство  мобильный  производитель  управление
5 значение  переменный  массив  строка
6 браузер  веб  страница  библиотека
7 программа  компьютер  процессор  программный
8 продукт  мобильный  рынок  дизайн
9 сеть  интернет  трафик  связь
10 сайт  страница  домен  контент
11 тест  тестирование  ошибка  библиотека
12 объект  свойство  свет  пространство
13 запрос  таблица  ответ  страница
14 модель  изображение  обучение  алгоритм
15 сервис  облачный  сообщение  платформа
16 язык  программирование  программист  библиотека
17 бот  сообщение  канал  игрок
18 рубль  российский  цена  россия
19 элемент  компонент  свойство  состояние
20 книга  читать  часы  электронный
21 сотрудник  бизнес  рабочий  документ
22 класс  метод  библиотека  реализация
23 виртуальный  машина  диск  память
24 конференция  участник  видео  тема
25 центр  дата  оборудование  инфраструктур

In [33]:
vectorizer = TfidfVectorizer(max_features=800, min_df=17, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(str_texts)
model = NMF(n_components=40)
model.fit(X)
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 вообще  думать  жизнь  ничто
1 игра  игрок  игровой  играть
2 файл  папка  скрипт  строка
3 элемент  значение  компонент  блок
4 устройство  смартфон  мобильный  телефон
5 продукт  мобильный  дизайн  инструмент
6 космический  аппарат  земля  станция
7 язык  программирование  программист  программа
8 сеть  интернет  трафик  связь
9 сервер  домен  настройка  адрес
10 доклад  конференция  тема  участник
11 сайт  страница  браузер  веб
12 тест  тестирование  тестовый  ошибка
13 запрос  таблица  строка  ответ
14 сигнал  частота  диапазон  канал
15 бот  скрипт  канал  друг
16 изображение  обучение  алгоритм  модель
17 класс  метод  библиотека  реализация
18 база  база дать  таблица  запись
19 печать  мм  материал  температура
20 товар  скидка  магазин  цена
21 дата центр  центр  дата  оборудование
22 сотрудник  бизнес  ит  рабочий
23 книга  часы  профессиональный  читать
24 робот  ребёнок  датчик  движение
25 учёный  мозг  исследование  научный
26 виртуальный  машина  реальность  резервный

In [34]:
vectorizer = CountVectorizer(max_features=100, min_df=5, max_df=0.4, ngram_range=(1,4))
X = vectorizer.fit_transform(str_texts)
model = NMF(n_components=20)
model.fit(X)
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 код  ошибка  программа  тест
1 игра  разработчик  разработка  знать
2 тип  класс  метод  интерфейс
3 значение  состояние  строка  день
4 элемент  список  страница  запись
5 устройство  управление  уровень  настройка
6 файл  модуль  добавить  параметр
7 приложение  версия  разработчик  разработка
8 сайт  страница  строка  интернет
9 сервер  клиент  сервис  настройка
10 функция  параметр  уровень  строка
11 сеть  интерфейс  связь  интернет
12 модель  метод  уровень  модуль
13 точка  находиться  скорость  область
14 язык  программа  слово  технология
15 команда  разработка  разработчик  программа
16 запрос  база  запись  сервис
17 объект  метод  класс  инструмент
18 ваш  продукт  говорить  клиент
19 число  программа  строка  далее


**9. Темы лучшей NMF модели. Сравнение с LDA**

На мой взгляд, лучше всего справилась последняя модель.

Геймдев
> 1 игра  разработчик  разработка  уровень

Тестирование
> 0 код  ошибка  тест  написать

Возможно речь идет о настройке сервиса 
> 9 сервер  клиент  сервис  настройка

Показалось, что по сравнению с выделенными темами в LDA модели NMF справляется лучше. 